In [1]:
# the following model is an adjusted version of the one from the link
# https://medium.com/@sauravsharma_19630/action-recognition-with-inbuilt-pytorch-features-ac2280d1c338
# the adjustments where made to on the one hand be able to run on Azure and on the other had to generate a smoking detecto
# instead of a general activity recognizer

In [ ]:
# Install dependencies for video module in pytorch to work
# ! sudo apt-get install libavformat-dev libavdevice-dev
# ! pip install av==6.2.0

In [2]:
# Import required modules ...

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader
from torch.optim.lr_scheduler import StepLR
import torchvision
from torchvision import get_video_backend
from torchvision.models.video import r3d_18 
from torchvision import transforms
import os
from tqdm.auto import tqdm
import numpy as np
import time
import av
import random
print(f"PyAV version -- {av.__version__}")

SEED = 491
torch.manual_seed(SEED)

from collections import OrderedDict
import warnings
warnings.filterwarnings('ignore')

#run_av_diagnostics()

PyAV version -- 6.2.0


In [2]:
# diagnostics for PyAV installation. For now version 6.2.0 works 
def run_av_diagnostics():
    import av
    av.open("video_data/brush_hair/Aussie_Brunette_Brushing_Hair_II_brush_hair_u_nm_np1_ba_goo_4.avi")
    print(get_video_backend())
    av.logging.set_level(av.logging.ERROR)
    if not hasattr(av.video.frame.VideoFrame, 'pict_type'):
      print("Nah")

run_av_diagnostics()

pyav


In [3]:
# memory footprint support libraries/code
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 55.7 GB  |     Proc size: 340.6 MB
GPU RAM Free: 12206MB | Used: 0MB | Util   0% | Total     12206MB


In [4]:
#the following transformations are from this github file
#https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
def crop(vid, i, j, h, w):
    return vid[..., i:(i + h), j:(j + w)]


def center_crop(vid, output_size):
    h, w = vid.shape[-2:]
    th, tw = output_size

    i = int(round((h - th) / 2.))
    j = int(round((w - tw) / 2.))
    return crop(vid, i, j, th, tw)


def hflip(vid):
    return vid.flip(dims=(-1,))


# NOTE: for those functions, which generally expect mini-batches, we keep them
# as non-minibatch so that they are applied as if they were 4d (thus image).
# this way, we only apply the transformation in the spatial domain
def resize(vid, size, interpolation='bilinear'):
    # NOTE: using bilinear interpolation because we don't work on minibatches
    # at this level
    scale = None
    if isinstance(size, int):
        scale = float(size) / min(vid.shape[-2:])
        size = None
    return torch.nn.functional.interpolate(
        vid, size=size, scale_factor=scale, mode=interpolation, align_corners=False)


def pad(vid, padding, fill=0, padding_mode="constant"):
    # NOTE: don't want to pad on temporal dimension, so let as non-batch
    # (4d) before padding. This works as expected
    return torch.nn.functional.pad(vid, padding, value=fill, mode=padding_mode)


def to_normalized_float_tensor(vid):
    return vid.permute(3, 0, 1, 2).to(torch.float32) / 255


def normalize(vid, mean, std):
    shape = (-1,) + (1,) * (vid.dim() - 1)
    mean = torch.as_tensor(mean).reshape(shape)
    std = torch.as_tensor(std).reshape(shape)
    return (vid - mean) / std


# Class interface

class RandomCrop(object):
    def __init__(self, size):
        self.size = size

    @staticmethod
    def get_params(vid, output_size):
        """Get parameters for ``crop`` for a random crop.
        """
        h, w = vid.shape[-2:]
        th, tw = output_size
        if w == tw and h == th:
            return 0, 0, h, w
        i = random.randint(0, h - th)
        j = random.randint(0, w - tw)
        return i, j, th, tw

    def __call__(self, vid):
        i, j, h, w = self.get_params(vid, self.size)
        return crop(vid, i, j, h, w)


class CenterCrop(object):
    def __init__(self, size):
        self.size = size

    def __call__(self, vid):
        return center_crop(vid, self.size)


class Resize(object):
    def __init__(self, size):
        self.size = size

    def __call__(self, vid):
        return resize(vid, self.size)


class ToFloatTensorInZeroOne(object):
    def __call__(self, vid):
        return to_normalized_float_tensor(vid)


class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, vid):
        return normalize(vid, self.mean, self.std)


class RandomHorizontalFlip(object):
    def __init__(self, p=0.5):
        self.p = p

    def __call__(self, vid):
        if random.random() < self.p:
            return hflip(vid)
        return vid


class Pad(object):
    def __init__(self, padding, fill=0):
        self.padding = padding
        self.fill = fill

    def __call__(self, vid):
        return pad(vid, self.padding, self.fill)


In [5]:
class AverageMeter(object):
    """
    Computes and stores the average and current value
    """
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def update_acc(self, val, n=1):
        self.val = val/n
        self.sum += val
        self.count += n
        self.avg = self.sum / self.count

In [6]:
# define model using PyTorch
class VideoRecog_Model(nn.Module):
  def __init__(self):
      super(VideoRecog_Model, self).__init__()
      self.base_model = nn.Sequential(*list(r3d_18(pretrained=True).children())[:-1])
      self.fc1 = nn.Linear(512, 250)
      self.fc2 = nn.Linear(250, 1) 
      self.dropout = nn.Dropout2d(0.3)
       

  def forward(self, x):
      out = self.base_model(x).squeeze(4).squeeze(3).squeeze(2) # output of base model is bs x 512 x 1 x 1 x 1
      out = F.relu(self.fc1(out)) 
      out = self.dropout(out) 

      out=self.fc2(out)
      
      return out

In [7]:
def train(config, model, loader, optimizer, epoch):
    model.train()
    config = {}
    config['log_interval'] = 100
    correct = 0
    total_loss = 0.0
    flag = 0
    Loss, Acc = AverageMeter(), AverageMeter()
    start = time.time()
    for batch_id, data in enumerate(loader):
        data, target = data[0], data[-1]
        
        #reorganize dataset to only smoking or not
        # print("here")
        target = target.numpy()
        bs = len(target)
        target = [1 if t == 4 else 0 for t in target]
        target = torch.Tensor(target)
        
        if torch.cuda.is_available():
           data = data.cuda()
           target = target.cuda()

        optimizer.zero_grad()
        output = model(data)
        #reweighting the positive examples since unbalanced pos vs. negative
        # and trying to increase the recall -> rather false alarm than no alarm
        pos_weight = torch.Tensor([952/226 for t in range(bs)])
        if torch.cuda.is_available():
            pos_weight=pos_weight.cuda()
        crit = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        loss = crit(output.squeeze(), target.float())
        
        # loss = F.nll_loss(output, target)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()

        Loss.update(loss.item(), data.size(0))

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        num_corrects = pred.eq(target.view_as(pred)).sum().item()
        correct += num_corrects

        Acc.update_acc(num_corrects, data.size(0))

        if flag!= 0 and batch_id%config['log_interval'] == 0:
           print('Train Epoch: {} Batch [{}/{} ({:.0f}%)]\tLoss: {:.6f} Accuracy: {}/{} ({:.0f})%'.format(
                epoch, batch_id * len(data), len(loader.dataset),
                100. * batch_id / len(loader), Loss.avg, correct, Acc.count, 100. * Acc.avg))
        flag = 1

    #total_loss /= len(loader.dataset) 
    print('Train Epoch: {} Average Loss: {:.6f} Average Accuracy: {}/{} ({:.0f})%'.format(
         epoch, Loss.avg, correct, Acc.count, 100. * Acc.avg ))
    print(f"Takes {time.time() - start}")

In [8]:
def test(config, model, loader, text='Validation',show_case=False):
    model.eval()
    correct = 0
    total_loss = 0.0
    Loss, Acc = AverageMeter(), AverageMeter()
    with torch.no_grad():
         for batch_id, data in enumerate(loader):
             data, target = data[0], data[-1]
             target = target.numpy()

             #labeling for smokeing and non smokeing
             #since we have 5 different activity types in binary_folder we have to change the lables as following:
             #brush_hair 0 -> 0
             #catch 1 -> 0
             #draw_sword 2 -> 0
             #eat 3 -> 0
             #smoke 4 -> 1

             target = [1 if t == 4 else 0 for t in target]
             target = torch.Tensor(target)
             

             if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()


             output = model(data)
             if show_case:
                 print(nn.Sigmoid(output))
            
             crit = nn.BCEWithLogitsLoss()
             loss =crit(output.squeeze(), target.float())

             total_loss += loss.item()

             Loss.update(loss.item(), data.size(0))

             pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
             num_corrects = pred.eq(target.view_as(pred)).sum().item()
             correct += num_corrects

             Acc.update_acc(num_corrects, data.size(0))
           
    total_loss /= len(loader.dataset)
    print(text + ' Average Loss: {:.6f} Average Accuracy: {}/{} ({:.0f})%'.format(
         Loss.avg, correct, Acc.count , 100. * Acc.avg ))

In [9]:
# Datasets and Dataloaders for model training ..

val_split = 0.10
num_frames = 16 # 16
clip_steps = 50
num_workers = 6
pin_memory = True
train_tfms = torchvision.transforms.Compose([
                                 ToFloatTensorInZeroOne(),
                                 Resize((128, 171)),
                                 RandomHorizontalFlip(),
                                 Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 RandomCrop((112, 112))
                               ])  
test_tfms =  torchvision.transforms.Compose([
                                             ToFloatTensorInZeroOne(),
                                             Resize((128, 171)),
                                             Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                             CenterCrop((112, 112))
                                             ])
hmdb51_train = torchvision.datasets.HMDB51('data/binary_folder/', 'data/test_train_splits/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=True,
                                                transform=train_tfms, num_workers=num_workers)


hmdb51_test = torchvision.datasets.HMDB51('data/binary_folder/', 'data/test_train_splits/', num_frames,
                                                step_between_clips = clip_steps, fold=1, train=False,
                                                transform=test_tfms, num_workers=num_workers)
      
total_train_samples = len(hmdb51_train)
total_val_samples = round(val_split * total_train_samples)

print(f"number of train samples {total_train_samples}")
print(f"number of validation samples {total_val_samples}")
print(f"number of test samples {len(hmdb51_test)}")


  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

number of train samples 952
number of validation samples 95
number of test samples 412


In [16]:
torch.cuda.is_available()

True

In [11]:
bs = 6
lr = 1e-3
gamma = 0.7
total_epochs = 4
config = {}
num_workers = 0

kwargs = {'num_workers':num_workers, 'pin_memory':True} if torch.cuda.is_available() else {'num_workers':num_workers}
#kwargs = {'num_workers':num_workers}
#kwargs = {}

hmdb51_train_v1, hmdb51_val_v1 = random_split(hmdb51_train, [total_train_samples - total_val_samples,
                                                                       total_val_samples])

#hmdb51_train_v1.video_clips.compute_clips(16, 1, frame_rate=30)
#hmdb51_val_v1.video_clips.compute_clips(16, 1, frame_rate=30)
#hmdb51_test.video_clips.compute_clips(16, 1, frame_rate=30)

#train_sampler = RandomClipSampler(hmdb51_train_v1.video_clips, 5)
#test_sampler = UniformClipSampler(hmdb51_test.video_clips, 5)
  
train_loader = DataLoader(hmdb51_train_v1, batch_size=bs, shuffle=True, **kwargs)
val_loader   = DataLoader(hmdb51_val_v1, batch_size=bs, shuffle=True, **kwargs)
test_loader  = DataLoader(hmdb51_test, batch_size=bs, shuffle=False, **kwargs)

model = VideoRecog_Model()

if torch.cuda.is_available():
   model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

print("Launching Action Recognition Model training")
# train(config, model, train_loader, optimizer, epoch)
for epoch in range(1, total_epochs + 1):
    train(config, model, train_loader, optimizer, epoch)
    test(config, model, val_loader, text="Validation")
    scheduler.step()

test(config, model, test_loader, text="Test")

Launching Action Recognition Model training
Train Epoch: 1 Batch [600/857 (70%)]	Loss: 1.420198 Accuracy: 450/606 (74)%
Train Epoch: 1 Average Loss: 1.327465 Average Accuracy: 648/857 (76)%
Takes 333.0490663051605
Validation Average Loss: 0.947670 Average Accuracy: 78/95 (82)%
Train Epoch: 2 Batch [600/857 (70%)]	Loss: 1.083379 Accuracy: 464/606 (77)%
Train Epoch: 2 Average Loss: 1.090239 Average Accuracy: 648/857 (76)%
Takes 329.9412839412689
Validation Average Loss: 1.087228 Average Accuracy: 78/95 (82)%
Train Epoch: 3 Batch [600/857 (70%)]	Loss: 1.112954 Accuracy: 446/606 (74)%
Train Epoch: 3 Average Loss: 1.092549 Average Accuracy: 648/857 (76)%
Takes 329.3133499622345
Validation Average Loss: 0.654706 Average Accuracy: 78/95 (82)%
Train Epoch: 4 Batch [600/857 (70%)]	Loss: 1.042025 Accuracy: 465/606 (77)%
Train Epoch: 4 Average Loss: 1.052902 Average Accuracy: 648/857 (76)%
Takes 328.8185088634491
Validation Average Loss: 0.679460 Average Accuracy: 78/95 (82)%
Test Average Loss: 1